In [1]:
import os
import rootpath
import sys
sys.path.append(rootpath.detect())

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
from testsuite.results import Result, ResultsContainer
from testsuite.analysis_tools import strip_problem_names
from testsuite.utilities import Pareto_split

In [4]:
from table_utilities import save_table, get_comparison_matrices

In [7]:
def save_table(*args, **kwargs):
    pass
save_table(1, a=2)

In [8]:
from figure_utilities import PATH_TO_REPORT_REPO, PATH_TO_PAREGO_RESULTS

ImportError: cannot import name 'PATH_TO_PAREGO_RESULTS' from 'figure_utilities' (/home/finley/phd/code/dsaf/experiments/dsaf_vs_saf/analysis/results_analysis/figure_utilities.py)

In [6]:
assert os.path.isdir(PATH_TO_PROCESSED_RESULTS)

In [7]:
RESULT_FILES_LIST = sorted(os.listdir(PATH_TO_PROCESSED_RESULTS))

In [35]:
RESULT_FILES_LIST
UNATTAINABLE_RESULTS_FILE_LIST =  [str(file) for file in RESULT_FILES_LIST if int(file.split("_")[-1][:-5]) in [0, 3]]
PARETO_RESULTS_FILE_LIST =  [str(file) for file in RESULT_FILES_LIST if int(file.split("_")[-1][:-5]) in [1, 4]]
ATTAINABLE_RESULTS_FILE_LIST = [str(file) for file in RESULT_FILES_LIST if int(file.split("_")[-1][:-5]) in [2, 5]]

In [36]:
RESULT_FILES_LIST_2OBJ = [file for file in RESULT_FILES_LIST if strip_problem_names if strip_problem_names(file[:-7])[1] ==2]
RESULT_FILES_LIST_3OBJ = [file for file in RESULT_FILES_LIST if strip_problem_names if strip_problem_names(file[:-7])[1] ==3]
RESULT_FILES_LIST_4OBJ = [file for file in RESULT_FILES_LIST if strip_problem_names if strip_problem_names(file[:-7])[1] ==4]

In [37]:
def get_table_from_list(LIST, attainable=False):
    D = {}
    for file_name in LIST:
        key = file_name[:-7]
        
        if attainable:
            med, iqr, med_r, iqr_r = get_med_iqr_b(file_name)
        else:
            med, iqr, med_r, iqr_r = get_med_iqr_a(file_name)
        data0 = np.hstack((med.reshape(-1,1), iqr.reshape(-1,1)))
        data1 = np.hstack((med_r.reshape(-1,1), iqr_r.reshape(-1,1)))
        if key+"_1" not in D.keys(): 
            D[key+f"_1"] = np.hstack([di for di in data0]+[di for di in data1])
        else:
            D[key+f"_2"] = np.hstack([di for di in data0]+[di for di in data1])
    return D
ans = get_table_from_list(RESULT_FILES_LIST, attainable=False)

In [39]:
for k, v in ans.items():
    print(f"{k}:\t{v.round(3)}")

wfg1_2obj_3dim_1:	[0.546 0.032 0.549 0.036 0.549 0.036 0.551 0.034 0.558 0.039 0.558 0.039
 0.563 0.039 0.563 0.039]
wfg1_2obj_3dim_2:	[0.803 0.054 0.826 0.03  0.843 0.036 0.844 0.029 0.804 0.077 0.816 0.063
 0.822 0.054 0.824 0.055]


In [6]:
def apply_f_to_mask(df, mask, f):
    best_inds =  [*zip(*np.where(mask == 1))] 
    for i, j in best_inds:
        df.iloc[i, j] = f(df.iloc[i, j])

In [7]:
def format_table(df):
    df_str = df.astype(str)
    # italicise IQR columns
    mask_italics = np.zeros_like(df_str.values, dtype=bool)
    mask_italics[:,1::2] = True
    f_italics = lambda cell_value: r"\textit{"+cell_value+"}"
    apply_f_to_mask(df_str, mask_italics, f_italics)
    
    # get boolean mask of winners/same results for each
    values_directed = df_str.values[:,:int(df_str.values.shape[1]/2)]
    values_undirected = df_str.values[:,int(df_str.values.shape[1]/2):]
    
    # highlight best
    directed_wins = np.repeat((values_directed>values_undirected)[:, ::2], 2, axis=1)
    undirected_wins = np.repeat((values_directed<values_undirected)[:, ::2], 2, axis=1)
    mask_best = np.hstack([directed_wins, undirected_wins])
    f_best = lambda cell_value: r"\best "+cell_value
    apply_f_to_mask(df_str, mask_best, f_best)
    
    # highlight draws
    draw = np.repeat((values_directed==values_undirected)[:, ::2], 2, axis=1)
    mask_draw = np.hstack([draw, draw])
    f_draw = lambda cell_value: r"\statsimilar "+cell_value
    apply_f_to_mask(df_str, mask_draw, f_draw)

    return df_str

In [21]:
def get_med_iqr_a(file_name):
    TABLE_STAGES = [25, 50, 100, 150]
    result = load_result(file_name)
    stage_inds = [np.where(np.asarray(result.dual_hpv_hist_x)==ts)[0][0] for ts in TABLE_STAGES] 
    
    med = np.median(result.dual_hpv_history[0][:, stage_inds], axis=0)
    lqr = np.quantile(result.dual_hpv_history[0][:, stage_inds], 0.25, axis=0)
    uqr = np.quantile(result.dual_hpv_history[0][:, stage_inds], 0.75, axis=0)
    
    med_r = np.median(result.dual_hpvref_history[0][:, stage_inds], axis=0)
    lqr_r = np.quantile(result.dual_hpvref_history[0][:, stage_inds], 0.25, axis=0)
    uqr_r = np.quantile(result.dual_hpvref_history[0][:, stage_inds], 0.75, axis=0)
    
    return med, uqr-lqr, med_r, uqr_r-lqr_r

def get_med_iqr_b(file_name):
    TABLE_STAGES = [25, 50, 100, 150]
    result = load_result(file_name)
    stage_inds = [np.where(np.asarray(result.dual_hpv_hist_x)==ts)[0][0] for ts in TABLE_STAGES] 
    
    med = np.median(result.dual_hpv_history[1][:, stage_inds], axis=0)
    lqr = np.quantile(result.dual_hpv_history[1][:, stage_inds], 0.25, axis=0)
    uqr = np.quantile(result.dual_hpv_history[1][:, stage_inds], 0.75, axis=0)
    
    med_r = np.median(result.dual_hpvref_history[1][:, stage_inds], axis=0)
    lqr_r = np.quantile(result.dual_hpvref_history[1][:, stage_inds], 0.25, axis=0)
    uqr_r = np.quantile(result.dual_hpvref_history[1][:, stage_inds], 0.75, axis=0)
    
    return med, uqr-lqr, med_r, uqr_r-lqr_r

def get_table_from_list(LIST, attainable=False):
    D = {}
    for file_name in LIST:
        key = file_name[:-7]
        
        if attainable:
            med, iqr, med_r, iqr_r = get_med_iqr_b(file_name)
        else:
            med, iqr, med_r, iqr_r = get_med_iqr_a(file_name)
        data0 = np.hstack((med.reshape(-1,1), iqr.reshape(-1,1)))
        data1 = np.hstack((med_r.reshape(-1,1), iqr_r.reshape(-1,1)))
        if key+"_1" not in D.keys(): 
            D[key+f"_1"] = np.hstack([di for di in data0]+[di for di in data1])
        else:
            D[key+f"_2"] = np.hstack([di for di in data0]+[di for di in data1])
    return D

In [9]:
# collated unattainable results
D = get_table_from_list(UNATTAINABLE_RESULTS_FILE_LIST)
mic = pd.MultiIndex.from_product([["Directed", "Undirected"], [25, 50, 100, 150],['median', 'IQR']])
mir = pd.MultiIndex.from_product([[d[:-2].replace("_", " ") for d in D.keys()][::2], ["1", "2"]], names=[r"$f$", "Position"])
df = pd.DataFrame.from_dict(D, orient="index", columns=mic).round(3)
df2 = pd.DataFrame(df.values, index=mir, columns=mic).round(3)
df2

df_unattainable = format_table(df2)
# save_table(df_unattainable, "unattainable_table")
df_unattainable

Directed                                     \
                                  25                                 50    
                               median                   IQR       median   
$f$             Position                                                   
wfg1 2obj 3dim  1               0.546        \textit{0.032}        0.549   
                2               0.666        \textit{0.041}  \best 0.713   
wfg1 3obj 4dim  1         \best 0.568  \best \textit{0.135}  \best 0.598   
                2               0.531         \textit{0.07}        0.553   
wfg1 4obj 5dim  1         \best 0.429   \best \textit{0.09}  \best 0.469   
                2         \best 0.423  \best \textit{0.081}   \best 0.48   
wfg2 2obj 6dim  1               0.871         \textit{0.06}  \best 0.923   
                2         \best 0.778  \best \textit{0.115}  \best 0.799   
wfg2 3obj 6dim  1               0.767        \textit{0.121}         0.81   
                2         \best 0.839  \best \textit{0.092}  \best 0.873   
wfg2 4obj 10dim 1         \best 0.654  \best \textit{0.101}  \best 0.693   
                2         \best 0.913  \best \textit{0.036}  \best 0.952   
wfg3 2obj 6dim  1               0.922        \textit{0.027}         0.95   
                2               0.954        \textit{0.029}        0.986   
wfg3 3obj 10dim 1         \best 0.892  \best \textit{0.036}  \best 0.936   
                2         \best 0.931  \best \textit{0.051}        0.972   
wfg3 4obj 10dim 1         \best 0.895  \best \textit{0.063}  \best 0.949   
                2         \best 0.919  \best \textit{0.059}  \best 0.976   
wfg4 2obj 6dim  1               0.854        \textit{0.138}  \best 0.918   
                2         \best 0.964   \best \textit{0.02}   \best 0.98   
wfg4 3obj 8dim  1         \best 0.874  \best \textit{0.108}  \best 0.951   
                2          \best 0.79  \best \textit{0.141}  \best 0.923   
wfg4 4obj 8dim  1         \best 0.698   \best \textit{0.13}  \best 0.849   
                2         \best 0.778  \best \textit{0.167}  \best 0.912   
wfg5 2obj 6dim  1          \best 0.91  \best \textit{0.075}  \best 0.983   
                2         \best 0.952  \best \textit{0.037}  \best 0.985   
wfg5 3obj 8dim  1         \best 0.792  \best \textit{0.154}  \best 0.963   
                2         \best 0.847    \best \textit{0.1}  \best 0.966   
wfg5 4obj 10dim 1         \best 0.566  \best \textit{0.113}  \best 0.772   
                2         \best 0.726  \best \textit{0.116}  \best 0.899   
wfg6 2obj 6dim  1         \best 0.914  \best \textit{0.053}  \best 0.978   
                2         \best 0.892  \best \textit{0.065}  \best 0.957   
wfg6 3obj 8dim  1         \best 0.812  \best \textit{0.116}  \best 0.953   
                2         \best 0.914  \best \textit{0.048}  \best 0.974   
wfg6 4obj 10dim 1               0.554        \textit{0.112}    \best 0.8   
                2         \best 0.731  \best \textit{0.101}  \best 0.854   

                                                             \
                                                        100   
                                           IQR       median   
$f$             Position                                      
wfg1 2obj 3dim  1               \textit{0.036}        0.549   
                2         \best \textit{0.056}  \best 0.716   
wfg1 3obj 4dim  1         \best \textit{0.136}        0.598   
                2               \textit{0.089}        0.554   
wfg1 4obj 5dim  1         \best \textit{0.078}  \best 0.472   
                2         \best \textit{0.081}    \best 0.5   
wfg2 2obj 6dim  1         \best \textit{0.058}  \best 0.959   
                2         \best \textit{0.102}  \best 0.847   
wfg2 3obj 6dim  1                \textit{0.13}  \best 0.856   
                2         \best \textit{0.065}   \best 0.91   
wfg2 4obj 10dim 1         \best \textit{0.096}        0.759   
                2          \best \textit{0.

In [10]:
# collated Pareto results
D = get_table_from_list(PARETO_RESULTS_FILE_LIST)
df = pd.DataFrame.from_dict(D, orient="index", columns=mic).round(3)
df2 = pd.DataFrame(df.values, index=mir, columns=mic).round(3)
df2

df_pareto = format_table(df2)
# save_table(df_pareto, "pareto_table")
df_pareto

Directed                                     \
                                  25                                 50    
                               median                   IQR       median   
$f$             Position                                                   
wfg1 2obj 3dim  1               0.551        \textit{0.049}        0.561   
                2               0.763         \textit{0.04}  \best 0.784   
wfg1 3obj 4dim  1         \best 0.584  \best \textit{0.095}  \best 0.619   
                2         \best 0.601  \best \textit{0.093}  \best 0.644   
wfg1 4obj 5dim  1         \best 0.447  \best \textit{0.096}  \best 0.473   
                2         \best 0.473  \best \textit{0.095}  \best 0.517   
wfg2 2obj 6dim  1         \best 0.923   \best \textit{0.06}  \best 0.959   
                2         \best 0.797  \best \textit{0.109}  \best 0.815   
wfg2 3obj 6dim  1               0.795        \textit{0.091}        0.832   
                2                0.85        \textit{0.131}  \best 0.898   
wfg2 4obj 10dim 1               0.636        \textit{0.084}  \best 0.697   
                2         \best 0.914  \best \textit{0.036}  \best 0.958   
wfg3 2obj 6dim  1         \best 0.959  \best \textit{0.029}  \best 0.993   
                2               0.976        \textit{0.018}  \best 0.997   
wfg3 3obj 10dim 1         \best 0.903  \best \textit{0.037}   \best 0.97   
                2         \best 0.947  \best \textit{0.031}        0.982   
wfg3 4obj 10dim 1         \best 0.916  \best \textit{0.046}  \best 0.972   
                2          \best 0.92  \best \textit{0.046}  \best 0.974   
wfg4 2obj 6dim  1         \best 0.881  \best \textit{0.108}  \best 0.943   
                2         \best 0.984  \best \textit{0.009}  \best 0.994   
wfg4 3obj 8dim  1         \best 0.853   \best \textit{0.14}  \best 0.963   
                2         \best 0.844  \best \textit{0.175}  \best 0.932   
wfg4 4obj 8dim  1         \best 0.671  \best \textit{0.266}  \best 0.881   
                2          \best 0.82  \best \textit{0.121}  \best 0.927   
wfg5 2obj 6dim  1         \best 0.941  \best \textit{0.063}  \best 0.988   
                2         \best 0.976  \best \textit{0.021}  \best 0.995   
wfg5 3obj 8dim  1         \best 0.786  \best \textit{0.133}   \best 0.98   
                2         \best 0.907   \best \textit{0.05}  \best 0.983   
wfg5 4obj 10dim 1         \best 0.602  \best \textit{0.148}  \best 0.765   
                2         \best 0.756  \best \textit{0.141}  \best 0.923   
wfg6 2obj 6dim  1         \best 0.931  \best \textit{0.068}  \best 0.975   
                2         \best 0.908   \best \textit{0.08}  \best 0.976   
wfg6 3obj 8dim  1         \best 0.804  \best \textit{0.128}   \best 0.97   
                2         \best 0.917  \best \textit{0.063}   \best 0.98   
wfg6 4obj 10dim 1           \best 0.6  \best \textit{0.117}  \best 0.807   
                2         \best 0.761  \best \textit{0.077}  \best 0.912   

                                                             \
                                                        100   
                                           IQR       median   
$f$             Position                                      
wfg1 2obj 3dim  1               \textit{0.056}        0.566   
                2         \best \textit{0.031}  \best 0.794   
wfg1 3obj 4dim  1         \best \textit{0.096}        0.626   
                2         \best \textit{0.081}        0.654   
wfg1 4obj 5dim  1           \best \textit{0.1}  \best 0.488   
                2         \best \textit{0.062}  \best 0.529   
wfg2 2obj 6dim  1         \best \textit{0.043}  \best 0.989   
                2         \best \textit{0.105}  \best 0.884   
wfg2 3obj 6dim  1               \textit{0.109}  \best 0.878   
                2         \best \textit{0.112}  \best 0.923   
wfg2 4obj 10dim 1         \best \textit{0.107}  \best 0.795   
                2         \best \textit{0.0

In [11]:
# collated unattainable results
D = get_table_from_list(ATTAINABLE_RESULTS_FILE_LIST)
df = pd.DataFrame.from_dict(D, orient="index", columns=mic).round(3)
df2 = pd.DataFrame(df.values, index=mir, columns=mic).round(3)
df2

df_attainable_a = format_table(df2)
# save_table(df_attainable_a, "attainable_table_volA")
df_attainable_a

Directed                             \
                                       25                               
                                    median                        IQR   
$f$             Position                                                
wfg1 2obj 3dim  1                    0.586             \textit{0.053}   
                2                    0.803             \textit{0.054}   
wfg1 3obj 4dim  1              \best 0.595       \best \textit{0.121}   
                2                     0.58             \textit{0.126}   
wfg1 4obj 5dim  1              \best 0.464       \best \textit{0.104}   
                2              \best 0.485       \best \textit{0.079}   
wfg2 2obj 6dim  1                    0.951             \textit{0.033}   
                2              \best 0.839       \best \textit{0.141}   
wfg2 3obj 6dim  1                    0.803             \textit{0.164}   
                2                    0.861             \textit{0.097}   
wfg2 4obj 10dim 1              \best 0.656       \best \textit{0.116}   
                2               \best 0.92       \best \textit{0.042}   
wfg3 2obj 6dim  1              \best 0.999       \best \textit{0.008}   
                2                \best 1.0       \best \textit{0.005}   
wfg3 3obj 10dim 1              \best 0.953       \best \textit{0.049}   
                2              \best 0.972       \best \textit{0.028}   
wfg3 4obj 10dim 1              \best 0.949       \best \textit{0.042}   
                2              \best 0.968        \best \textit{0.03}   
wfg4 2obj 6dim  1              \best 0.919       \best \textit{0.178}   
                2         \statsimilar 1.0  \statsimilar \textit{0.0}   
wfg4 3obj 8dim  1                \best 0.9       \best \textit{0.165}   
                2              \best 0.911       \best \textit{0.149}   
wfg4 4obj 8dim  1              \best 0.683       \best \textit{0.121}   
                2              \best 0.843       \best \textit{0.131}   
wfg5 2obj 6dim  1              \best 0.988       \best \textit{0.054}   
                2                \best 1.0       \best \textit{0.011}   
wfg5 3obj 8dim  1              \best 0.871       \best \textit{0.184}   
                2               \best 0.95        \best \textit{0.06}   
wfg5 4obj 10dim 1              \best 0.583        \best \textit{0.11}   
                2               \best 0.77       \best \textit{0.116}   
wfg6 2obj 6dim  1              \best 0.971       \best \textit{0.061}   
                2               \best 0.98        \best \textit{0.06}   
wfg6 3obj 8dim  1              \best 0.886       \best \textit{0.142}   
                2              \best 0.959       \best \textit{0.029}   
wfg6 4obj 10dim 1              \best 0.603       \best \textit{0.114}   
                2              \best 0.757       \best \textit{0.132}   

                                                                       \
                                       50                               
                                    median                        IQR   
$f$             Position                                                
wfg1 2obj 3dim  1              \best 0.598        \best \textit{0.07}   
                2              \best 0.826        \best \textit{0.03}   
wfg1 3obj 4dim  1                      0.6              \textit{0.12}   
                2              \best 0.651       \best \textit{0.112}   
wfg1 4obj 5dim  1              \best 0.479       \best \textit{0.081}   
                2              \best 0.529       \best \textit{0.079}   
wfg2 2obj 6dim  1              \best 0.985       \best \textit{0.016}   
                2              \best 0.848       \best \textit{0.134}   
wfg2 3obj 6dim  1                    0.841             \textit{0.136}   
                2                    0.902             \textit{0.063}   
wfg2 4obj 10dim 1                    0.679             \textit{0.104}   
                2 

In [12]:
# collated attainable results, volume B
D = get_table_from_list(ATTAINABLE_RESULTS_FILE_LIST, attainable=True)
df = pd.DataFrame.from_dict(D, orient="index", columns=mic).round(3)
df2 = pd.DataFrame(df.values, index=mir, columns=mic).round(3)
df2

df_attainable_b = format_table(df2)
# save_table(df_attainable_b, "attainable_table_volB")
df_attainable_b

Directed                               \
                                       25                                 
                                    median                          IQR   
$f$             Position                                                  
wfg1 2obj 3dim  1         \statsimilar 0.0    \statsimilar \textit{0.0}   
                2         \statsimilar 0.0    \statsimilar \textit{0.0}   
wfg1 3obj 4dim  1         \statsimilar 0.0    \statsimilar \textit{0.0}   
                2         \statsimilar 0.0    \statsimilar \textit{0.0}   
wfg1 4obj 5dim  1         \statsimilar 0.0    \statsimilar \textit{0.0}   
                2         \statsimilar 0.0    \statsimilar \textit{0.0}   
wfg2 2obj 6dim  1         \statsimilar 0.0    \statsimilar \textit{0.0}   
                2         \statsimilar 0.0    \statsimilar \textit{0.0}   
wfg2 3obj 6dim  1         \statsimilar 0.0    \statsimilar \textit{0.0}   
                2         \statsimilar 0.0    \statsimilar \textit{0.0}   
wfg2 4obj 10dim 1         \statsimilar 0.0    \statsimilar \textit{0.0}   
                2         \statsimilar 0.0    \statsimilar \textit{0.0}   
wfg3 2obj 6dim  1         \statsimilar 0.0  \statsimilar \textit{0.025}   
                2         \statsimilar 0.0   \statsimilar \textit{0.06}   
wfg3 3obj 10dim 1         \statsimilar 0.0    \statsimilar \textit{0.0}   
                2         \statsimilar 0.0    \statsimilar \textit{0.0}   
wfg3 4obj 10dim 1         \statsimilar 0.0    \statsimilar \textit{0.0}   
                2         \statsimilar 0.0    \statsimilar \textit{0.0}   
wfg4 2obj 6dim  1         \statsimilar 0.0    \statsimilar \textit{0.0}   
                2              \best 0.089         \best \textit{0.141}   
wfg4 3obj 8dim  1         \statsimilar 0.0    \statsimilar \textit{0.0}   
                2         \statsimilar 0.0    \statsimilar \textit{0.0}   
wfg4 4obj 8dim  1         \statsimilar 0.0    \statsimilar \textit{0.0}   
                2         \statsimilar 0.0    \statsimilar \textit{0.0}   
wfg5 2obj 6dim  1         \statsimilar 0.0    \statsimilar \textit{0.0}   
                2              \best 0.105         \best \textit{0.288}   
wfg5 3obj 8dim  1         \statsimilar 0.0    \statsimilar \textit{0.0}   
                2         \statsimilar 0.0    \statsimilar \textit{0.0}   
wfg5 4obj 10dim 1         \statsimilar 0.0    \statsimilar \textit{0.0}   
                2         \statsimilar 0.0    \statsimilar \textit{0.0}   
wfg6 2obj 6dim  1         \statsimilar 0.0    \statsimilar \textit{0.0}   
                2         \statsimilar 0.0    \statsimilar \textit{0.0}   
wfg6 3obj 8dim  1         \statsimilar 0.0    \statsimilar \textit{0.0}   
                2         \statsimilar 0.0    \statsimilar \textit{0.0}   
wfg6 4obj 10dim 1         \statsimilar 0.0    \statsimilar \textit{0.0}   
                2         \statsimilar 0.0    \statsimilar \textit{0.0}   

                                                                         \
                                       50                                 
                                    median                          IQR   
$f$             Position                                                  
wfg1 2obj 3dim  1         \statsimilar 0.0    \statsimilar \textit{0.0}   
                2         \statsimilar 0.0    \statsimilar \textit{0.0}   
wfg1 3obj 4dim  1         \statsimilar 0.0    \statsimilar \textit{0.0}   
                2         \statsimilar 0.0    \statsimilar \textit{0.0}   
wfg1 4obj 5dim  1         \statsimilar 0.0    \statsimilar \textit{0.0}   
                2         \statsimilar 0.0    \statsimilar \textit{0.0}   
wfg2 2obj 6dim  1         \statsimilar 0.0    \statsimilar \textit{0.0}   
                2         \statsimilar 0.0    \statsimilar \textit{0.0}   
wfg2 3obj 6dim  1         \statsimilar 0.0    \statsimilar \textit{0.0}   
                2         \statsimilar 0.0    \statsimilar

### RANKED SCORES

In [13]:
def get_comparison_matrices(result, intervals=None):
    """
    
    """
    if intervals is None:
        ra = result.dual_hpv_history[0]
        ra_ref = result.dual_hpvref_history[0]
        rb = result.dual_hpv_history[1]
        rb_ref = result.dual_hpvref_history[1]
        
    else:
        assert isinstance(intervals, list), "intervals must be a list. For a single interval n supply [n]"
        assert all([i in result.dual_hpv_hist_x for i in intervals]), "Not all supplied intervals exist in results"
        
        inds = [np.where(np.asarray(result.dual_hpv_hist_x)==i)[0][0] for i in intervals]
        ra = result.dual_hpv_history[0][:, inds]
        ra_ref = result.dual_hpvref_history[0][:, inds]
        rb = result.dual_hpv_history[1][:, inds]
        rb_ref = result.dual_hpvref_history[1][:, inds]
        
    r = ra+rb
    r_ref = ra_ref+rb_ref
    M = (r>r_ref)
    M_ref = (r<r_ref)
    M_draw = (r==r_ref)
    return M, M_ref, M_draw

In [14]:
def add_gs_mask_to_df(df, mask):
    new_df = df.copy()
    for i in range(df.shape[0]):
        for j in range(df.shape[1]):
            new_df.iloc[i, j] = r"\cellcolor[gray]{"+str(mask[i, j])+"} " +new_df.iloc[i, j]
    return new_df

def grayscale_df_mask(df):
    desired_max = 0.6
    scaled_values = df.values.copy()
    scaled_values[scaled_values<0.5] = 0.5
    
    scaled_values = (scaled_values-0.5)*2
    
    return (1-(scaled_values*desired_max)).round(3)

In [15]:
def save_table(df, file_name, index=True):
    savedirs = [
        os.path.join(rootpath.detect(), "experiments/directed/analysis/results_tables/paper_figure_notebooks/tables/"),
        os.path.join(PATH_TO_REPORT_REPO, "tables/")]
    for d in savedirs:
        file_path = os.path.join(d, file_name+".tex")
        with open(file_path, "w") as outfile:
            print(df.to_latex(index=index, escape=False), file=outfile)

In [16]:
# def get_rank_table_from_list(names):
#     pairs = {0: 0,
#              1: 1,
#              2: 2,
#              3: 0, 
#              4: 1, 
#              5: 2}
#     # pair target locations
#     unattinable = get_rank_column_from_list([i for i in names if i.split("_")[-1][:-5] in ["0", "3"]])
#     pareto = get_rank_column_from_list([i for i in names if i.split("_")[-1][:-5] in ["1", "4"]])
#     attainable = get_rank_column_from_list([i for i in names if i.split("_")[-1][:-5] in ["2", "5"]])
#     ans = np.vstack((unattinable, pareto, attainable)).T
#     return ans
    

# def get_rank_column_from_list(LIST):
#     D = [] 
#     SCORES = []
#     for file_name in LIST:
#         key = file_name[:-7]
#         score = get_rank_score(file_name)
#         D.append(score)
#     return np.mean(D, axis=0)


In [17]:
def get_rank_table_from_list(LIST):
    D = {}
    for file_name in LIST:
        key = file_name[:-5]
        score = get_rank_score(file_name)
        D[key] = score 
    return D

def get_rank_score(file_name):
    TABLE_STAGES = [25, 50, 100, 150]
    result = load_result(file_name)
    rank_score = rank_comparison(result, intervals=TABLE_STAGES)
    return rank_score
    
    
def rank_comparison(resuts_container, intervals):
    M, M_draw = get_comparison_matrices(resuts_container, intervals=intervals)
    score_1 = M.astype(float)*1
    score_2 = M_draw.astype(float)*.5
    return (score_1+score_2).mean(axis=0)

        
def get_comparison_matrices(result, intervals=None, attainable=False):
    """
    
    """
    if intervals is None:
        ra = result.dual_hpv_history[0]
        ra_ref = result.dual_hpvref_history[0]
        rb = result.dual_hpv_history[1]
        rb_ref = result.dual_hpvref_history[1]
    else:
        assert isinstance(intervals, list), "intervals must be a list. For a single interval n supply [n]"
        assert all([i in result.dual_hpv_hist_x for i in intervals]), "Not all supplied intervals exist in results"
        inds = [np.where(np.asarray(result.dual_hpv_hist_x)==i)[0][0] for i in intervals]
        ra = result.dual_hpv_history[0][:, inds]
        ra_ref = result.dual_hpvref_history[0][:, inds]
        rb = result.dual_hpv_history[1][:, inds]
        rb_ref = result.dual_hpvref_history[1][:, inds]
        
    r = ra+rb
    r_ref = ra_ref+rb_ref
    c = (r>r_ref)
    u = np.logical_and(r==r_ref, r>0.)
    
    return c, u

def format_rank_df(df):
    cols = df.columns
    new_cols = []
    for i in range(cols.shape[0]):
        new_cols.append(list(cols[i]))
        for j in range(np.shape(new_cols[i])[0]):
            if new_cols[i][j][0].lower() == "w":
                new_cols[i][j] = new_cols[i][j].upper() 
    
    df_new = pd.DataFrame(df.round(3).values, columns=pd.MultiIndex.from_tuples(new_cols), index=df.index)
#     df_new =df_new.rename(columns = {old: new for old, new in zip(cols, new_cols)}).round(3)
    
    # define win
    mask_win = (df_new.values>0.5)
    f_win = lambda value: "\\best "+value
    
    # define_grayscale
    gs_mask = grayscale_df_mask(df_new) 
    
    # define draw 
    mask_draw = (df_new.values==0.5)
    f_draw = lambda value: "\\statsimilar "+value
    
    # apply masks
    df_new = df_new.astype(str)
#     apply_f_to_mask(df_new, mask_win, f_win)
#     apply_f_to_mask(df_new, mask_draw, f_draw)
    return add_gs_mask_to_df(df_new, gs_mask)

In [18]:
pair = {"0": "0",
        "1": "1",
        "2": "2",
        "3": "0", 
        "4": "1", 
        "5": "2"}

f_names = [i.split("_")[0] for i in RESULT_FILES_LIST]
f_obj = [i.split("_")[1] for i in RESULT_FILES_LIST]
f_dim = [i.split("_")[2] for i in RESULT_FILES_LIST]
f_t = [i.split("_")[-1][:-5] for i in RESULT_FILES_LIST]
f_attain = [pair[i] for i in f_t]

In [19]:
col_ind = pd.MultiIndex.from_tuples(zip(f_names, f_obj, f_dim, f_t, f_attain))

In [20]:
# overall ranked table
D = get_rank_table_from_list(RESULT_FILES_LIST)
df = pd.DataFrame.from_dict(D)
df_rank_raw = pd.DataFrame(df.values, columns=col_ind,  index=["25", "50", "100", "150"])
df_rank_raw.columns.names = ["f_names", "f_obj", "f_dim", "f_t", "f_attain"]
df_rank_raw

f_names       wfg1                                                    \
f_obj         2obj                                                     
f_dim         3dim                                                     
f_t              0         1         2         3         4         5   
f_attain         0         1         2         0         1         2   
25        0.419355  0.354839  0.387097  0.516129  0.612903  0.580645   
50        0.483871  0.387097  0.451613  0.548387  0.661290  0.580645   
100       0.483871  0.387097  0.451613  0.516129  0.661290  0.548387   
150       0.483871  0.419355  0.483871  0.580645  0.677419  0.580645   

f_names                                           ...      wfg6            \
f_obj         3obj                                ...      3obj             
f_dim         4dim                                ...      8dim             
f_t              0         1         2         3  ...         2         3   
f_attain         0         1         2         0  ...         2         0   
25        0.516129  0.645161  0.548387  0.387097  ...  0.870968  0.870968   
50        0.612903  0.645161  0.596774  0.322581  ...  1.000000  0.774194   
100       0.516129  0.612903  0.451613  0.354839  ...  0.967742  0.645161   
150       0.548387  0.645161  0.387097  0.387097  ...  1.000000  0.677419   

f_names                                                               \
f_obj                             4obj                                 
f_dim                            10dim                                 
f_t              4         5         0         1         2         3   
f_attain         1         2         0         1         2         0   
25        0.612903  0.709677  0.677419  0.806452  0.580645  0.741935   
50        0.612903  0.774194  0.935484  0.774194  0.903226  0.838710   
100       0.709677  0.967742  0.967742  0.967742  1.000000  0.903226   
150       0.612903  1.000000  0.967742  0.967742  0.967742  0.935484   

f_names                       
f_obj                         
f_dim                         
f_t              4         5  
f_attain         1         2  
25        0.806452  0.709677  
50        0.967742  0.967742  
100       1.000000  1.000000  
150       1.000000  0.903226  

[4 rows x 108 columns]

In [21]:
df_rank_raw.iloc[:, df_rank_raw.columns.get_level_values(0)=='wfg4']

f_names       wfg4                                                    \
f_obj         2obj                                                     
f_dim         6dim                                                     
f_t              0         1         2         3         4         5   
f_attain         0         1         2         0         1         2   
25        0.419355  0.580645  0.483871  0.516129  0.709677  0.774194   
50        0.387097  0.516129  0.548387  0.677419  0.774194  0.709677   
100       0.419355  0.225806  0.225806  0.629032  0.822581  0.806452   
150       0.387097  0.258065  0.161290  0.725806  0.822581  0.806452   

f_names                                                               \
f_obj         3obj                                                     
f_dim         8dim                                                     
f_t              0         1         2         3         4         5   
f_attain         0         1         2         0         1         2   
25        0.677419  0.580645  0.548387  0.516129  0.677419  0.612903   
50        0.709677  0.612903  0.677419  0.677419  0.564516  0.548387   
100       0.774194  0.612903  0.548387  0.677419  0.483871  0.451613   
150       0.806452  0.645161  0.677419  0.645161  0.387097  0.548387   

f_names                                                               
f_obj         4obj                                                    
f_dim         8dim                                                    
f_t              0         1         2         3         4         5  
f_attain         0         1         2         0         1         2  
25        0.677419  0.677419  0.612903  0.709677  0.677419  0.677419  
50        0.677419  0.677419  0.516129  0.677419  0.612903  0.709677  
100       0.612903  0.709677  0.612903  0.806452  0.677419  0.709677  
150       0.612903  0.677419  0.612903  0.774194  0.774194  0.709677

In [22]:
df_rank_overall = df_rank_raw.groupby(level=[0,1,2,4], axis=1).mean()
df_rank_overall

f_names       wfg1                                                    \
f_obj         2obj                          3obj                       
f_dim         3dim                          4dim                       
f_attain         0         1         2         0         1         2   
25        0.467742  0.483871  0.483871  0.451613  0.580645  0.500000   
50        0.516129  0.524194  0.516129  0.467742  0.564516  0.524194   
100       0.500000  0.524194  0.500000  0.435484  0.532258  0.419355   
150       0.532258  0.548387  0.532258  0.467742  0.532258  0.370968   

f_names                                     wfg2  ...      wfg5      wfg6  \
f_obj         4obj                          2obj  ...      4obj      2obj   
f_dim         5dim                          6dim  ...     10dim      6dim   
f_attain         0         1         2         0  ...         2         0   
25        0.548387  0.629032  0.564516  0.500000  ...  0.774194  0.854839   
50        0.580645  0.612903  0.596774  0.620968  ...  0.790323  0.741935   
100       0.516129  0.645161  0.612903  0.806452  ...  0.967742  0.645161   
150       0.596774  0.629032  0.612903  0.854839  ...  1.000000  0.677419   

f_names                                                               \
f_obj                             3obj                          4obj   
f_dim                             8dim                         10dim   
f_attain         1         2         0         1         2         0   
25        0.725806  0.790323  0.838710  0.677419  0.790323  0.709677   
50        0.709677  0.846774  0.854839  0.741935  0.887097  0.887097   
100       0.701613  0.830645  0.774194  0.822581  0.967742  0.935484   
150       0.645161  0.758065  0.822581  0.806452  1.000000  0.951613   

f_names                       
f_obj                         
f_dim                         
f_attain         1         2  
25        0.806452  0.645161  
50        0.870968  0.935484  
100       0.983871  1.000000  
150       0.983871  0.935484  

[4 rows x 54 columns]

In [23]:
df_rank_unattainable = df_rank_overall.iloc[:, ::3] 
df_rank_pareto = df_rank_overall.iloc[:, 1::3] 
df_rank_attainable = df_rank_overall.iloc[:, 2::3] 
df_rank_attainable

f_names       wfg1                          wfg2                      \
f_obj         2obj      3obj      4obj      2obj      3obj      4obj   
f_dim         3dim      4dim      5dim      6dim      6dim     10dim   
f_attain         2         2         2         2         2         2   
25        0.483871  0.500000  0.564516  0.604839  0.435484  0.483871   
50        0.516129  0.524194  0.596774  0.709677  0.548387  0.629032   
100       0.500000  0.419355  0.612903  0.741935  0.596774  0.467742   
150       0.532258  0.370968  0.612903  0.806452  0.661290  0.435484   

f_names       wfg3                          wfg4                      \
f_obj         2obj      3obj      4obj      2obj      3obj      4obj   
f_dim         6dim     10dim     10dim      6dim      8dim      8dim   
f_attain         2         2         2         2         2         2   
25        0.854839  0.758065  0.854839  0.629032  0.580645  0.645161   
50        0.887097  0.903226  0.935484  0.629032  0.612903  0.612903   
100       0.903226  1.000000  0.983871  0.516129  0.500000  0.661290   
150       0.790323  1.000000  1.000000  0.483871  0.612903  0.661290   

f_names       wfg5                          wfg6                      
f_obj         2obj      3obj      4obj      2obj      3obj      4obj  
f_dim         6dim      8dim     10dim      6dim      8dim     10dim  
f_attain         2         2         2         2         2         2  
25        0.790323  0.758065  0.774194  0.790323  0.790323  0.645161  
50        0.790323  0.919355  0.790323  0.846774  0.887097  0.935484  
100       0.903226  0.983871  0.967742  0.830645  0.967742  1.000000  
150       0.919355  1.000000  1.000000  0.758065  1.000000  0.935484

In [24]:
#2obj
df_wfg_2obj = df_rank_overall.iloc[:, df_rank_overall.columns.get_level_values(1)=='2obj'].groupby(level=[0, 1], axis=1).mean()
df_wfg_3obj = df_rank_overall.iloc[:, df_rank_overall.columns.get_level_values(1)=='3obj'].groupby(level=[0, 1], axis=1).mean()
df_wfg_4obj = df_rank_overall.iloc[:, df_rank_overall.columns.get_level_values(1)=='4obj'].groupby(level=[0, 1], axis=1).mean()

In [25]:
# totals by objective
df_wfg_totals = pd.concat([df_wfg_2obj.mean(axis=1), df_wfg_3obj.mean(axis=1), df_wfg_4obj.mean(axis=1)], axis=1)
df_wfg_totals.columns = ["\nobj = 2", "\nobj = 3", "\nobj = 4"]

df_wfg_totals["All"] = df_wfg_totals.mean(axis=1)
# df_wfg_totals = format_rank_df(df_wfg_totals)
df_wfg_totals

,\nobj = 2,\nobj = 3,\nobj = 4,All
25,0.635305,0.638889,0.674731,0.649642
50,0.667563,0.676523,0.737455,0.693847
100,0.674731,0.693548,0.775986,0.714755
150,0.682348,0.706093,0.771505,0.719982


In [26]:
# totals by attainability 
df_wfg_attainablility = pd.DataFrame(index=df_wfg_4obj.index)
df_wfg_attainablility["$A$"] = df_rank_unattainable.mean(axis=1).values
df_wfg_attainablility["$P$"] = df_rank_pareto.mean(axis=1).values
df_wfg_attainablility["$U$"] = df_rank_attainable.mean(axis=1).values

# df_wfg_attainablility = format_rank_df(df_wfg_attainablility) 
df_wfg_attainablility

,$A$,$P$,$U$
25,0.625448,0.659946,0.663530
50,0.661738,0.682348,0.737455
100,0.664875,0.726254,0.753136
150,0.665323,0.740143,0.754480


In [27]:
# combined M = 2 & 3
df_wfg_23obj = pd.concat([df_wfg_2obj, df_wfg_3obj], axis=1)

df_wfg_23obj.columns = pd.MultiIndex.from_tuples(list(zip((["\\nobj = 2"]*6)+(["\\nobj = 3"]*6), [i[0] for i in df_wfg_23obj.columns])))
df_wfg_23obj.index = ["25", "50", "100", "150"] 
# df_wfg_23obj.index.names= ["\\optstep"]
df_wfg_23obj = format_rank_df(df_wfg_23obj)
df_wfg_23obj.columns.names= ["", "\optstep"]
# save_table(df_wfg_23obj, "summary_table_01_2obj3obj")
df_wfg_23obj

\nobj = 2                                 \
\optstep                           WFG1                           WFG2   
25          \cellcolor[gray]{1.0} 0.478  \cellcolor[gray]{0.939} 0.551   
50        \cellcolor[gray]{0.977} 0.519  \cellcolor[gray]{0.794} 0.672   
100        \cellcolor[gray]{0.99} 0.508  \cellcolor[gray]{0.674} 0.772   
150       \cellcolor[gray]{0.954} 0.538   \cellcolor[gray]{0.62} 0.817   

                                                                        \
\optstep                           WFG3                           WFG4   
25        \cellcolor[gray]{0.826} 0.645  \cellcolor[gray]{0.903} 0.581   
50        \cellcolor[gray]{0.864} 0.613  \cellcolor[gray]{0.878} 0.602   
100       \cellcolor[gray]{0.839} 0.634  \cellcolor[gray]{0.974} 0.522   
150       \cellcolor[gray]{0.878} 0.602  \cellcolor[gray]{0.968} 0.527   

                                                                        \
\optstep                           WFG5                           WFG6   
25        \cellcolor[gray]{0.681} 0.766   \cellcolor[gray]{0.652} 0.79   
50          \cellcolor[gray]{0.6} 0.833  \cellcolor[gray]{0.681} 0.766   
100       \cellcolor[gray]{0.536} 0.887  \cellcolor[gray]{0.729} 0.726   
150         \cellcolor[gray]{0.5} 0.917  \cellcolor[gray]{0.767} 0.694   

                              \nobj = 3                                 \
\optstep                           WFG1                           WFG2   
25        \cellcolor[gray]{0.987} 0.511    \cellcolor[gray]{1.0} 0.484   
50        \cellcolor[gray]{0.977} 0.519  \cellcolor[gray]{0.994} 0.505   
100         \cellcolor[gray]{1.0} 0.462  \cellcolor[gray]{0.929} 0.559   
150         \cellcolor[gray]{1.0} 0.457   \cellcolor[gray]{0.87} 0.608   

                                                                        \
\optstep                           WFG3                           WFG4   
25        \cellcolor[gray]{0.755} 0.704  \cellcolor[gray]{0.878} 0.602   
50        \cellcolor[gray]{0.794} 0.672  \cellcolor[gray]{0.842} 0.632   
100       \cellcolor[gray]{0.755} 0.704  \cellcolor[gray]{0.891} 0.591   
150        \cellcolor[gray]{0.78} 0.683  \cellcolor[gray]{0.858} 0.618   

                                                                        
\optstep                           WFG5                           WFG6  
25        \cellcolor[gray]{0.684} 0.763  \cellcolor[gray]{0.677} 0.769  
50        \cellcolor[gray]{0.516} 0.903  \cellcolor[gray]{0.606} 0.828  
100       \cellcolor[gray]{0.413} 0.989  \cellcolor[gray]{0.574} 0.855  
150       \cellcolor[gray]{0.406} 0.995  \cellcolor[gray]{0.549} 0.876

In [29]:
# combined M = 2 & 3: split 1
df_wfg_23obj_1 = pd.concat([df_wfg_2obj], axis=1)
df_wfg_23obj_1.columns = pd.MultiIndex.from_tuples(list(zip((["\\nobj = 2"]*6), [i[0] for i in df_wfg_23obj_1.columns])))
df_wfg_23obj_1.index = ["25", "50", "100", "150"] 
# df_wfg_23obj_1.index.names= ["\\optstep"]
df_wfg_23obj_1 = format_rank_df(df_wfg_23obj_1)
df_wfg_23obj_1.columns.names= ["", "$\optstep$"]
# save_table(df_wfg_23obj_1, "summary_table_01_2obj3obj_1")
df_wfg_23obj_1

\nobj = 2                                 \
$\optstep$                           WFG1                           WFG2   
25            \cellcolor[gray]{1.0} 0.478  \cellcolor[gray]{0.939} 0.551   
50          \cellcolor[gray]{0.977} 0.519  \cellcolor[gray]{0.794} 0.672   
100          \cellcolor[gray]{0.99} 0.508  \cellcolor[gray]{0.674} 0.772   
150         \cellcolor[gray]{0.954} 0.538   \cellcolor[gray]{0.62} 0.817   

                                                                          \
$\optstep$                           WFG3                           WFG4   
25          \cellcolor[gray]{0.826} 0.645  \cellcolor[gray]{0.903} 0.581   
50          \cellcolor[gray]{0.864} 0.613  \cellcolor[gray]{0.878} 0.602   
100         \cellcolor[gray]{0.839} 0.634  \cellcolor[gray]{0.974} 0.522   
150         \cellcolor[gray]{0.878} 0.602  \cellcolor[gray]{0.968} 0.527   

                                                                          
$\optstep$                           WFG5                           WFG6  
25          \cellcolor[gray]{0.681} 0.766   \cellcolor[gray]{0.652} 0.79  
50            \cellcolor[gray]{0.6} 0.833  \cellcolor[gray]{0.681} 0.766  
100         \cellcolor[gray]{0.536} 0.887  \cellcolor[gray]{0.729} 0.726  
150           \cellcolor[gray]{0.5} 0.917  \cellcolor[gray]{0.767} 0.694

In [30]:
# combined M = 2 & 3: split 2
df_wfg_23obj_2 = pd.concat([df_wfg_3obj], axis=1)
df_wfg_23obj_2.columns = pd.MultiIndex.from_tuples(list(zip((["\\nobj = 3"]*6), [i[0] for i in df_wfg_23obj_2.columns])))
df_wfg_23obj_2.index = ["25", "50", "100", "150"] 
# df_wfg_23obj_2.index.names= ["\\optstep"]
df_wfg_23obj_2 = format_rank_df(df_wfg_23obj_2) 
df_wfg_23obj_2.columns.names= ["", "$\optstep$"]
# save_table(df_wfg_23obj_2, "summary_table_01_2obj3obj_2", index=False)
df_wfg_23obj_2

\nobj = 3                                 \
$\optstep$                           WFG1                           WFG2   
25          \cellcolor[gray]{0.987} 0.511    \cellcolor[gray]{1.0} 0.484   
50          \cellcolor[gray]{0.977} 0.519  \cellcolor[gray]{0.994} 0.505   
100           \cellcolor[gray]{1.0} 0.462  \cellcolor[gray]{0.929} 0.559   
150           \cellcolor[gray]{1.0} 0.457   \cellcolor[gray]{0.87} 0.608   

                                                                          \
$\optstep$                           WFG3                           WFG4   
25          \cellcolor[gray]{0.755} 0.704  \cellcolor[gray]{0.878} 0.602   
50          \cellcolor[gray]{0.794} 0.672  \cellcolor[gray]{0.842} 0.632   
100         \cellcolor[gray]{0.755} 0.704  \cellcolor[gray]{0.891} 0.591   
150          \cellcolor[gray]{0.78} 0.683  \cellcolor[gray]{0.858} 0.618   

                                                                          
$\optstep$                           WFG5                           WFG6  
25          \cellcolor[gray]{0.684} 0.763  \cellcolor[gray]{0.677} 0.769  
50          \cellcolor[gray]{0.516} 0.903  \cellcolor[gray]{0.606} 0.828  
100         \cellcolor[gray]{0.413} 0.989  \cellcolor[gray]{0.574} 0.855  
150         \cellcolor[gray]{0.406} 0.995  \cellcolor[gray]{0.549} 0.876

In [31]:
df_wfg_4Tobj = pd.concat([df_wfg_4obj, df_wfg_attainablility, df_wfg_totals], axis=1)
df_wfg_4Tobj.index = ["25", "50", "100", "150"] 
df_wfg_4Tobj.columns = pd.MultiIndex.from_tuples(list(zip((["\\nobj = 4"]*6)+(["Overall"]*7), [i[0] if isinstance(i, tuple) else i for i in df_wfg_4Tobj.columns ])))
# df_wfg_23obj.index.names= ["\\optstep"]
df_wfg_4Tobj = format_rank_df(df_wfg_4Tobj) 
df_wfg_4Tobj.columns.names= ["", "$\optstep$"]

# save_table(df_wfg_4Tobj, "summary_table_02_4objTotal")
df_wfg_4Tobj

\nobj = 4                                 \
$\optstep$                           WFG1                           WFG2   
25          \cellcolor[gray]{0.903} 0.581  \cellcolor[gray]{0.948} 0.543   
50          \cellcolor[gray]{0.884} 0.597  \cellcolor[gray]{0.851} 0.624   
100         \cellcolor[gray]{0.891} 0.591  \cellcolor[gray]{0.981} 0.516   
150         \cellcolor[gray]{0.864} 0.613    \cellcolor[gray]{1.0} 0.462   

                                                                          \
$\optstep$                           WFG3                           WFG4   
25          \cellcolor[gray]{0.658} 0.785  \cellcolor[gray]{0.794} 0.672   
50          \cellcolor[gray]{0.555} 0.871  \cellcolor[gray]{0.826} 0.645   
100          \cellcolor[gray]{0.49} 0.925  \cellcolor[gray]{0.774} 0.688   
150         \cellcolor[gray]{0.503} 0.914  \cellcolor[gray]{0.767} 0.694   

                                                                          \
$\optstep$                           WFG5                           WFG6   
25          \cellcolor[gray]{0.704} 0.747   \cellcolor[gray]{0.736} 0.72   
50           \cellcolor[gray]{0.652} 0.79  \cellcolor[gray]{0.522} 0.898   
100         \cellcolor[gray]{0.446} 0.962  \cellcolor[gray]{0.432} 0.973   
150         \cellcolor[gray]{0.413} 0.989  \cellcolor[gray]{0.452} 0.957   

                                  Overall                                 \
$\optstep$                            $A$                            $P$   
25           \cellcolor[gray]{0.85} 0.625   \cellcolor[gray]{0.808} 0.66   
50          \cellcolor[gray]{0.806} 0.662  \cellcolor[gray]{0.782} 0.682   
100         \cellcolor[gray]{0.802} 0.665  \cellcolor[gray]{0.729} 0.726   
150         \cellcolor[gray]{0.802} 0.665   \cellcolor[gray]{0.712} 0.74   

                                                                          \
$\optstep$                            $U$                      \nobj = 2   
25          \cellcolor[gray]{0.803} 0.664  \cellcolor[gray]{0.838} 0.635   
50          \cellcolor[gray]{0.716} 0.737  \cellcolor[gray]{0.798} 0.668   
100         \cellcolor[gray]{0.696} 0.753   \cellcolor[gray]{0.79} 0.675   
150         \cellcolor[gray]{0.695} 0.754  \cellcolor[gray]{0.782} 0.682   

                                                                          \
$\optstep$                      \nobj = 3                      \nobj = 4   
25          \cellcolor[gray]{0.833} 0.639   \cellcolor[gray]{0.79} 0.675   
50          \cellcolor[gray]{0.788} 0.677  \cellcolor[gray]{0.716} 0.737   
100         \cellcolor[gray]{0.767} 0.694  \cellcolor[gray]{0.669} 0.776   
150         \cellcolor[gray]{0.753} 0.706  \cellcolor[gray]{0.674} 0.772   

                                           
$\optstep$                            All  
25            \cellcolor[gray]{0.82} 0.65  
50          \cellcolor[gray]{0.767} 0.694  
100         \cellcolor[gray]{0.742} 0.715  
150          \cellcolor[gray]{0.736} 0.72

In [32]:
df_wfg_4Tobj_1 = pd.concat([df_wfg_4obj], axis=1)
df_wfg_4Tobj_1.index = ["25", "50", "100", "150"] 
df_wfg_4Tobj_1.columns = pd.MultiIndex.from_tuples(list(zip((["\\nobj = 4"]*6)+(["Overall"]*7), [i[0] for i in df_wfg_4Tobj_1.columns])))
# df_wfg_23obj.index.names= ["\\optstep"]
df_wfg_4Tobj_1 = format_rank_df(df_wfg_4Tobj_1)
df_wfg_4Tobj_1.columns.names= ["", "$\optstep$"]

# save_table(df_wfg_4Tobj_1, "summary_table_02_4objTotal_1")
df_wfg_4Tobj_1

\nobj = 4                                 \
$\optstep$                           WFG1                           WFG2   
25          \cellcolor[gray]{0.903} 0.581  \cellcolor[gray]{0.948} 0.543   
50          \cellcolor[gray]{0.884} 0.597  \cellcolor[gray]{0.851} 0.624   
100         \cellcolor[gray]{0.891} 0.591  \cellcolor[gray]{0.981} 0.516   
150         \cellcolor[gray]{0.864} 0.613    \cellcolor[gray]{1.0} 0.462   

                                                                          \
$\optstep$                           WFG3                           WFG4   
25          \cellcolor[gray]{0.658} 0.785  \cellcolor[gray]{0.794} 0.672   
50          \cellcolor[gray]{0.555} 0.871  \cellcolor[gray]{0.826} 0.645   
100          \cellcolor[gray]{0.49} 0.925  \cellcolor[gray]{0.774} 0.688   
150         \cellcolor[gray]{0.503} 0.914  \cellcolor[gray]{0.767} 0.694   

                                                                          
$\optstep$                           WFG5                           WFG6  
25          \cellcolor[gray]{0.704} 0.747   \cellcolor[gray]{0.736} 0.72  
50           \cellcolor[gray]{0.652} 0.79  \cellcolor[gray]{0.522} 0.898  
100         \cellcolor[gray]{0.446} 0.962  \cellcolor[gray]{0.432} 0.973  
150         \cellcolor[gray]{0.413} 0.989  \cellcolor[gray]{0.452} 0.957

In [33]:
df_wfg_4Tobj_2 = pd.concat([df_wfg_attainablility, df_wfg_totals], axis=1)
df_wfg_4Tobj_2.index = ["25", "50", "100", "150"] 
df_wfg_4Tobj_2.columns = pd.MultiIndex.from_tuples(list(zip((["Overall"]*7), df_wfg_4Tobj_2.columns)))
# df_wfg_23obj.index.names= ["\\optstep"]
df_wfg_4Tobj_2 = format_rank_df(df_wfg_4Tobj_2)
df_wfg_4Tobj_2.columns.names= ["", "$\optstep$"]
# df_wfg_4Tobj_2.columns.names= ["", ""]


# save_table(df_wfg_4Tobj_2, "summary_table_02_4objTotal_2", index=False)
df_wfg_4Tobj_2

Overall                                 \
$\optstep$                            $A$                            $P$   
25           \cellcolor[gray]{0.85} 0.625   \cellcolor[gray]{0.808} 0.66   
50          \cellcolor[gray]{0.806} 0.662  \cellcolor[gray]{0.782} 0.682   
100         \cellcolor[gray]{0.802} 0.665  \cellcolor[gray]{0.729} 0.726   
150         \cellcolor[gray]{0.802} 0.665   \cellcolor[gray]{0.712} 0.74   

                                                                          \
$\optstep$                            $U$                      \nobj = 2   
25          \cellcolor[gray]{0.803} 0.664  \cellcolor[gray]{0.838} 0.635   
50          \cellcolor[gray]{0.716} 0.737  \cellcolor[gray]{0.798} 0.668   
100         \cellcolor[gray]{0.696} 0.753   \cellcolor[gray]{0.79} 0.675   
150         \cellcolor[gray]{0.695} 0.754  \cellcolor[gray]{0.782} 0.682   

                                                                          \
$\optstep$                      \nobj = 3                      \nobj = 4   
25          \cellcolor[gray]{0.833} 0.639   \cellcolor[gray]{0.79} 0.675   
50          \cellcolor[gray]{0.788} 0.677  \cellcolor[gray]{0.716} 0.737   
100         \cellcolor[gray]{0.767} 0.694  \cellcolor[gray]{0.669} 0.776   
150         \cellcolor[gray]{0.753} 0.706  \cellcolor[gray]{0.674} 0.772   

                                           
$\optstep$                            All  
25            \cellcolor[gray]{0.82} 0.65  
50          \cellcolor[gray]{0.767} 0.694  
100         \cellcolor[gray]{0.742} 0.715  
150          \cellcolor[gray]{0.736} 0.72

In [34]:
n_prob = 1
n_obj = 3
names = [ i for i in RESULT_FILES_LIST if strip_problem_names(i[:-7])[0:2] == (n_prob, n_obj)]
rs = [load_result(n) for n in names]
C = []
D = []
U = []
ANS = []
INDS = [25, 50, 100, 150]
for i in range(6):
    x = [np.where(np.array(rs[i].dual_hpv_hist_x) == index)[0][0] for index in INDS]
    a = rs[i].dual_hpv_history[0][:, x]
    a_ref = rs[i].dual_hpvref_history[0][:, x]
    
    b = rs[i].dual_hpv_history[1][:, x]
    b_ref = rs[i].dual_hpvref_history[1][:, x]
    
    total = a+b
    total_ref = a_ref+b_ref 
    c = (total>total_ref).mean(axis=0)
    
    u_t = np.logical_and(total==total_ref, total>0.).astype(float)
    u = u_t.mean(axis=0)*0.5
    C.append(c+u)
    